##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# feels.like with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

# Additional Data Sources

DeepFashion (http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html)

@inproceedings{liuLQWTcvpr16DeepFashion,
 author = {Liu, Ziwei and Luo, Ping and Qiu, Shi and Wang, Xiaogang and Tang, Xiaoou},
 title = {DeepFashion: Powering Robust Clothes Recognition and Retrieval with Rich Annotations},
 booktitle = {Proceedings of IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
 month = {June},
 year = {2016} 
 }

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [2]:
!pip install tflite-model-maker-nightly

     |████████████████████████████████| 593kB 29.8MB/s 
     |████████████████████████████████| 174kB 49.9MB/s 
     |████████████████████████████████| 645kB 36.5MB/s 
     |████████████████████████████████| 122kB 49.6MB/s 
     |████████████████████████████████| 1.1MB 47.4MB/s 
     |████████████████████████████████| 1.2MB 35.9MB/s 
     |████████████████████████████████| 112kB 56.2MB/s 
     |████████████████████████████████| 92kB 14.3MB/s 
     |████████████████████████████████| 686kB 43.5MB/s 
     |████████████████████████████████| 849kB 40.1MB/s 
     |████████████████████████████████| 6.3MB 49.8MB/s 
     |████████████████████████████████| 194kB 57.9MB/s 
     |████████████████████████████████| 686kB 45.5MB/s 
     |████████████████████████████████| 358kB 56.6MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 38.2MB 63kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=f400f72faf2744b2d09c

Import the required packages.

In [3]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

In [4]:
import os
os.getcwd()

'/content'

In [5]:
import shutil
shutil.unpack_archive("fashionimgs.zip")
image_path = os.path.join(os.path.dirname('fashionimgs.zip'), 'fashionimgs')

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [6]:
data = DataLoader.from_folder(image_path)
#80/10/10 split
train_data, rest_data = data.split(0.8)
validationdata, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 175, num_label: 13, labels: 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90.


2. Customize the TensorFlow model.

In [9]:
#define hyperparameters
epoch = 20
dropout = 0.3

In [10]:
model = image_classifier.create(train_data, epochs=epoch, dropout_rate=dropout, validation_data=validationdata, shuffle=True)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_1 (HubK (None, 1280)              3413024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                16653     
Total params: 3,429,677
Trainable params: 16,653
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


4/4 [==============================] - 2s 132ms/step - loss: 2.7827 - accuracy: 0.0781
Epoch 2/20
4/4 [==============================] - 0s 135ms/step - loss: 2.5284 - accuracy: 0.1641
Epoch 3/20
4/4 [==============================] - 1s 149ms/step - loss: 2.4119 - accuracy: 0.1797
Epoch 4/20
4/4 [==============================] - 0s 134ms/step - loss: 2.1657 - accuracy: 0.2578
Epoch 5/20
4/4 [==============================] - 0s 142ms/step - loss: 2.0498 - accuracy: 0.3125
Epoch 6/20
4/4 [==============================] - 0s 134ms/step - loss: 1.8903 - accuracy: 0.3672
Epoch 7/20
4/4 [==============================] - 0s 129ms/step - loss: 1.7929 - accuracy: 0.4766
Epoch 8/20
4/4 [==============================] - 0s 131ms/step - loss: 1.6907 - accuracy: 0.5391
Epoch 9/20
4/4 [==============================] - 0s 106ms/step - loss: 1.6721 - accuracy: 0.5781
Epoch 10/20
4/4 [==============================] - 0s 107ms/step - loss: 1.5171 - accuracy: 0.7031
Epoch 11/20
4/4 [=============

3. Evaluate the model.

In [11]:
loss, accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 2.0938 - accuracy: 0.3889


4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [ ]:
model.export(export_dir='.', with_metadata=False)

After this simple 4 steps, we can now download the model and label files, and continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android/#4).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).
